In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pygal 
from pygal import Config
from pywaffle import Waffle
from flask import Flask
from pygal.style import Style

%matplotlib inline

In [2]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
df = pd.read_csv(url, error_bad_lines=False)

In [4]:
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [5]:
df_maine = df[df.state == 'Maine']

In [6]:
df_maine.date.max()

'2020-04-02'

In [7]:
df_maine.county.unique()

array(['Androscoggin', 'Cumberland', 'Lincoln', 'Unknown', 'Kennebec',
       'Oxford', 'York', 'Penobscot', 'Sagadahoc', 'Knox', 'Waldo',
       'Franklin', 'Somerset', 'Hancock'], dtype=object)

In [8]:
df_state_tot = df_maine.groupby('date').sum()

In [9]:
df_state_tot.head()

,fips,cases,deaths
date,,,
2020-03-12,23001.0,1,0
2020-03-13,46006.0,2,0
2020-03-14,46006.0,3,0
2020-03-15,69021.0,12,0
2020-03-16,69021.0,17,0


df_state_tot = df_state_tot.head().reset_index()

In [10]:
dates = df_state_tot.index.values.tolist()
cases = df_state_tot.cases.values.tolist()
deaths = df_state_tot.deaths.values.tolist()

In [11]:
dates_major = dates[0::3]

## Plot Total Cases and Deaths

In [12]:
bar_chart = pygal.Bar(x_label_rotation=20, show_minor_x_labels=False, show_legend=False, y_title = 'Number of Cases')
bar_chart.title = 'Total COVID-19 Cases in Maine'
bar_chart.x_labels = dates
bar_chart.x_labels_major = dates_major
bar_chart.add('Number of Cases', cases)

bar_chart.render_to_file('plots/total_cases_maine.svg')                          # Save the svg to a file

In [13]:
bar_chart = pygal.Bar(x_label_rotation=20, show_minor_x_labels=False, show_legend=False, y_title='Number of Deaths')
bar_chart.title = 'Total COVID-19 Deaths in Maine'
bar_chart.x_labels = dates
bar_chart.x_labels_major = dates_major

bar_chart.add('Deaths', deaths)

bar_chart.render_to_file('plots/total_deaths_maine.svg')                          # Save the svg to a file

## Plot New Daily Cases

In [14]:
df_state_tot['new_cases'] = df_state_tot.cases.diff()
df_state_tot['new_cases'][0] = 1

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
df_state_tot.head()

,fips,cases,deaths,new_cases
date,,,,
2020-03-12,23001.0,1,0,1.0
2020-03-13,46006.0,2,0,1.0
2020-03-14,46006.0,3,0,1.0
2020-03-15,69021.0,12,0,9.0
2020-03-16,69021.0,17,0,5.0


In [16]:
bar_chart = pygal.Bar(x_label_rotation=20, 
                      show_minor_x_labels=False,
                      show_legend=False,
                      y_title = 'Number of New Cases',
                      x_title = 'Day')
bar_chart.title = 'Daily New COVID-19 Cases in Maine'
bar_chart.x_labels = dates
bar_chart.x_labels_major = dates_major
bar_chart.add('Number of New Cases', df_state_tot.new_cases.to_list())

bar_chart.render_to_file('plots/new_cases_maine.svg')                          # Save the svg to a file

## Plot Daily Deaths

In [17]:
df_state_tot['new_deaths'] = df_state_tot.deaths.diff()
df_state_tot['new_deaths'][0] = 0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
df_state_tot.tail()

,fips,cases,deaths,new_cases,new_deaths
date,,,,,
2020-03-29,276194.0,253,3,40.0,2.0
2020-03-30,276194.0,275,4,22.0,1.0
2020-03-31,276194.0,303,5,28.0,1.0
2020-04-01,299203.0,344,7,41.0,2.0
2020-04-02,299203.0,376,7,32.0,0.0


In [19]:
bar_chart = pygal.Bar(x_label_rotation=20, 
                      show_minor_x_labels=False,
                      show_legend=False,
                      y_title = 'Number of Deaths',
                      x_title = 'Day')
bar_chart.title = 'Daily COVID-19 Deaths in Maine'
bar_chart.x_labels = dates
bar_chart.x_labels_major = dates_major
bar_chart.add('Number of New Deaths', df_state_tot.new_deaths.to_list())

bar_chart.render_to_file('plots/new_deaths_maine.svg')                          # Save the svg to a file

## Plot County-Level Data

In [37]:
df_maine_today = df_maine[df_maine.date == df_maine.date.max()]

In [38]:
df_maine_today

,date,county,state,fips,cases,deaths
27088,2020-04-02,Androscoggin,Maine,23001.0,12,0
27089,2020-04-02,Cumberland,Maine,23005.0,204,5
27090,2020-04-02,Franklin,Maine,23007.0,3,0
27091,2020-04-02,Hancock,Maine,23009.0,2,0
27092,2020-04-02,Kennebec,Maine,23011.0,19,1
27093,2020-04-02,Knox,Maine,23013.0,7,0
27094,2020-04-02,Lincoln,Maine,23015.0,8,0
27095,2020-04-02,Oxford,Maine,23017.0,9,0
27096,2020-04-02,Penobscot,Maine,23019.0,16,0
27097,2020-04-02,Sagadahoc,Maine,23023.0,10,0


In [39]:
df_maine_today.sort_values(by=['cases'], ascending=False, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
bar_chart = pygal.Bar(x_label_rotation=20, show_legend=False, y_title='Number of Cases', x_title='County')
bar_chart.title = 'COVID-19 Cases by County (March 31, 2020)'
bar_chart.x_labels = df_maine_today.county.to_list()
bar_chart.add('Cases', df_maine_today.cases.to_list())

bar_chart.render_to_file('plots/current_cases_by_county.svg')                          # Save the svg to a file

##  County Case Growth

In [24]:
def create_days_to_double_data(df, days_to_double):
    cases= [1]
    n_days = len(df.date.unique())
    d = range(1,len(df.date.unique()))
    for day in d:
        cases.append(round(2**(day/days_to_double),2))
    return cases

In [25]:
def get_custom_style():
    custom_style = Style(
    colors=['#85144B', '#111111', '#7FDBFF', '#39CCCC', '#3D9970', '#2ECC40', '#01FF70',
            '#FFDC00', '#FF851B', '#FF4136', '#F012BE', '#B10DC9', '#00008b', '#0074D9',
            '#6e6e6e', '#9e9e9e', '#dbdbdb'],
    label_font_size=14,
    major_guide_stroke_dasharray= '1.5,1.5'
    )

    return custom_style

def case_by_county_config():
    config = Config()
    custom_style = get_custom_style()
    config.style=custom_style
    config.x_label_rotation=20
    config.show_minor_x_labels=False
    config.y_labels_major_every=3
    config.show_minor_y_labels=False
    config.truncate_legend=-1 

    return config

In [26]:
def plot_county_lines(df_maine, line_chart):
    for county in df_maine.county.unique():
        if len(list(df_maine.date.unique())) == len(df_maine[df_maine.county==county].cases):
            case_data = df_maine[df_maine.county==county].cases
        else:
            len_diff = len(list(df_maine.date.unique())) - len(df_maine[df_maine.county==county].cases)
            case_data = df_maine[df_maine.county==county].cases.to_list()
            case_data = [0]*len_diff + case_data
        line_chart.add(county, case_data, dots_size=1.5)

In [27]:
config = case_by_county_config()

line_chart = pygal.Line(config,
                        y_title='Number of Cases',
                        height=500,
                        width=700)
line_chart.title = 'COVID-19 Case Growth by County'
line_chart.x_labels = list(df_maine.date.unique())
line_chart.x_labels_major = list(df_maine.date.unique())[0::3]
#add a line for each county
plot_county_lines(df_maine, line_chart)

line_chart.render_to_file('plots/growth_by_county.svg')                          # Save the svg to a file

In [28]:
config = case_by_county_config()

line_chart = pygal.Line(config,
                        y_title='Cases',
                        logarithmic=True,
                        height=500,
                        width=800                       )
line_chart.title = 'COVID-19 Case Growth by County (log scale)'
line_chart.x_labels = list(df_maine.date.unique())
line_chart.x_labels_major = list(df_maine.date.unique())[0::3]

#add a line for each county
plot_county_lines(df_maine, line_chart)

ref_style = stroke_style={'width':2.5}
line_chart.add('Cases Double every 4 Days', create_days_to_double_data(df_maine, 4),
              stroke_style=ref_style, dots_size=1)
line_chart.add('Cases Double every 5 Days', create_days_to_double_data(df_maine, 5),
               stroke_style=ref_style, dots_size=1)
line_chart.add('Cases Double every Week', create_days_to_double_data(df_maine, 7),
               stroke_style=ref_style, dots_size=1)


line_chart.render_to_file('plots/growth_by_county_log.svg')                          # Save the svg to a file

###### Cumberland

In [29]:
bar_chart = pygal.Bar(x_label_rotation=20, show_legend=False, y_title='Number of Cases', show_minor_x_labels=False)
bar_chart.title = 'COVID-19 Cases in Cumberland County'
bar_chart.x_labels = df_maine[df_maine.county=='Cumberland'].date.to_list()
bar_chart.x_labels_major = df_maine[df_maine.county=='Cumberland'].date.to_list()[0::3]
bar_chart.add('Cases', df_maine[df_maine.county=='Cumberland'].cases.to_list())

bar_chart.render_to_file('plots/cumberland_cases.svg')                          # Save the svg to a file

#### York

In [30]:
bar_chart = pygal.Bar(x_label_rotation=20, show_legend=False, y_title='Number of Cases', show_minor_x_labels=False)
bar_chart.title = 'COVID-19 Cases in York County'
bar_chart.x_labels = df_maine[df_maine.county=='York'].date.to_list()
bar_chart.x_labels_major = df_maine[df_maine.county=='York'].date.to_list()[0::3]
bar_chart.add('Cases', df_maine[df_maine.county=='York'].cases.to_list())

bar_chart.render_to_file('plots/york_cases.svg')                          # Save the svg to a file

#### Kennebec

In [31]:
bar_chart = pygal.Bar(x_label_rotation=20, show_legend=False, y_title='Number of Cases', show_minor_x_labels=False)
bar_chart.title = 'COVID-19 Cases in Kennebec County'
bar_chart.x_labels = df_maine[df_maine.county=='Kennebec'].date.to_list()
bar_chart.x_labels_major = df_maine[df_maine.county=='Kennebec'].date.to_list()[0::3]
bar_chart.add('Cases', df_maine[df_maine.county=='Kennebec'].cases.to_list())

bar_chart.render_to_file('plots/kennebec_cases.svg')                          # Save the svg to a file

## Cases Per Capita

#### County Population

https://data.census.gov/cedsci/profile?g=0500000US23005&q=Cumberland
https://data.census.gov/cedsci/profile?g=0500000US23031&q=York
https://data.census.gov/cedsci/profile?g=0500000US23017&q=Oxford
https://data.census.gov/cedsci/profile?g=0500000US23023&q=Sagadahoc
https://data.census.gov/cedsci/profile?g=0500000US23001&q=Androscoggin
    https://data.census.gov/cedsci/profile?g=0500000US23015&q=Lincoln
        https://data.census.gov/cedsci/profile?g=0500000US23011&q=Kennebec
https://data.census.gov/cedsci/profile?g=0500000US23007&q=Franklin
    https://data.census.gov/cedsci/profile?g=0500000US23027&q=Waldo
        https://data.census.gov/cedsci/profile?g=0500000US23025&q=Somerset
            https://data.census.gov/cedsci/profile?g=0500000US23009&q=Hancock
https://data.census.gov/cedsci/profile?g=0500000US23029&q=Washington
https://data.census.gov/cedsci/profile?g=0500000US23021&q=Piscataquis
    https://data.census.gov/cedsci/profile?g=0500000US23019&q=Penobscot
        https://data.census.gov/cedsci/profile?g=0500000US23003&q=Aroostook

In [32]:
population_data = {'Cumberland': 290944,
                   'York':203102,
                   'Oxford':57325,
                   'Sagadahoc':35277,
                   'Androscoggin':107444,
                   'Lincoln':34067,
                   'Kennebec':121545,
                   'Franklin':30019,
                   'Knox':39823,
                   'Waldo':39418,
                   'Somerset':50710,
                   'Hancock':54541,
                   'Washington':31694,
                   'Piscataquis':16877,
                   'Penobscot':151748,
                   'Aroostook': 68269,
                   'Unknown':np.nan}

In [40]:
df_population = pd.DataFrame.from_dict(population_data, orient='index',columns=['population'])
df_maine_today = df_maine_today.merge(df_population, left_on='county', right_index=True)
df_maine_today['cases_per_ten_thousand'] = df_maine_today.cases/ (df_maine_today.population/10000)
df_maine_today = df_maine_today.round({'cases_per_ten_thousand':1})


,date,county,state,fips,cases,deaths,population,cases_per_ten_thousand
27089,2020-04-02,Cumberland,Maine,23005.0,204,5,290944.0,7.011659
27101,2020-04-02,York,Maine,23031.0,74,1,203102.0,3.643489
27092,2020-04-02,Kennebec,Maine,23011.0,19,1,121545.0,1.563207
27096,2020-04-02,Penobscot,Maine,23019.0,16,0,151748.0,1.054380
27088,2020-04-02,Androscoggin,Maine,23001.0,12,0,107444.0,1.116861


In [43]:
df_maine_today.head()

,date,county,state,fips,cases,deaths,population,cases_per_ten_thousand
27089,2020-04-02,Cumberland,Maine,23005.0,204,5,290944.0,7.0
27101,2020-04-02,York,Maine,23031.0,74,1,203102.0,3.6
27092,2020-04-02,Kennebec,Maine,23011.0,19,1,121545.0,1.6
27096,2020-04-02,Penobscot,Maine,23019.0,16,0,151748.0,1.1
27088,2020-04-02,Androscoggin,Maine,23001.0,12,0,107444.0,1.1


In [49]:
#Drop the Unknown county column
unknown_idx = df_maine_today[df_maine_today.county=='Unknown'].index
df_maine_today = df_maine_today.drop(labels=unknown_idx, axis=0)

In [35]:
bar_chart = pygal.Bar(x_label_rotation=20,
                      show_legend=False,
                      y_title='Cases per 100,000 Residents',
                      x_title='County')
bar_chart.title = 'COVID-19 Cases per 100,000 Residents (March 31, 2020)'
bar_chart.x_labels = df_maine_today.county.to_list()
bar_chart.add('Cases', df_maine_today.cases_per_hundred_thousand.to_list())

bar_chart.render_to_file('plots/current_cases_per_hundred_thous.svg')                          # Save the svg to a file

AttributeError: 'DataFrame' object has no attribute 'cases_per_hundred_thousand'

## Make a df with Press Herald Data (https://www.pressherald.com/2020/03/17/track-maines-coronavirus-cases-by-county/)


*press herald data used for recovered counts for March 26-31

In [ ]:
def append_recovered_data(df):
    recovered = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,24,36,41,41,68,80,94]
    if len(recovered) != len(df):
        recovered.append(np.nan)
    df['recovered'] = recovered
    return df

In [ ]:
df_state_tot = append_recovered_data(df_state_tot)
df_state_tot.tail()

In [ ]:
df_state_tot['active_cases'] = df_state_tot.cases - df_state_tot.deaths - df_state_tot.recovered

In [ ]:
bar_chart = pygal.StackedBar(x_label_rotation=20, show_minor_x_labels=False)
bar_chart.title = 'Maine COVID-19 Cases by Status'
bar_chart.x_labels = df_state_tot.index.values.tolist()
bar_chart.x_labels_major = df_state_tot.index.values.tolist()[0::3]

bar_chart.add('Active Cases', df_state_tot.active_cases.values.tolist())
bar_chart.add('Deaths', df_state_tot.deaths.values.tolist())
bar_chart.add('Recovered Cases', df_state_tot.recovered.values.tolist())


bar_chart.render_to_file('plots/case_status.svg')                          # Save the svg to a file

## Hospitalized Patients 

In [ ]:
df_state_tot['hospitalized']=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,49,57,63,68]

In [ ]:
df_state_tot.head()

In [ ]:
df_hosp = df_state_tot[df_state_tot.hospitalized > 0]
df_hosp

In [ ]:
bar_chart = pygal.Bar(x_label_rotation=20,
                      show_minor_x_labels=False,
                      show_legend=False,
                      y_title = 'Number of Hospitalized Patients')
bar_chart.title = 'Number of Patients Hospitalized for COVID-19 in Maine'
bar_chart.x_labels = df_hosp.index.values.tolist()
bar_chart.add('Count Hospitalized', df_hosp.hospitalized.values.tolist())

bar_chart.render_to_file('plots/hospitalized.svg')                          # Save the svg to a file

## Patient Age

In [ ]:
df_age = pd.DataFrame.from_dict({'age_range':['< 20','20s', '30s', '40s', '50s', '60s', '70s','80+'], 
                                 'cases': [9,43,35,67,87,96,58,37]})
df_age.head()

In [ ]:
total_count = df_age.cases.sum()
df_age['percent_of_tot'] = df_age.cases/total_count*100
df_age = df_age.round({'percent_of_tot':1})
df_age.head()

In [ ]:
fig = plt.figure(
    FigureClass=Waffle, 
    rows=4,
    values=df_age.percent_of_tot,
    labels=list(df_age.age_range),
    legend={'loc': 'lower left',
            'bbox_to_anchor': (-.01, -0.5),
            'ncol': len(df_age.percent_of_tot),
            'fontsize': 12,
            'title':'Age of Patient'
           },
    title={'label': 'Case Distribution by Patient Age',
           'loc': 'left',
           'pad':10,
           'fontdict': {'fontsize': 20}},
    figsize=(10, 8)  # figsize is a parameter of plt.figure

)

In [ ]:
bar_chart = pygal.Bar(x_label_rotation=20,
                      show_legend=False,
                      y_title='Percent of Cases (%)',
                      x_title='Age Group')
bar_chart.title = 'Case Distribution by Patient Age'
bar_chart.x_labels = df_age.age_range
bar_chart.add('% of Cases', df_age.percent_of_tot.to_list())

bar_chart.render_to_file('plots/age_range.svg')                          # Save the svg to a file